# Testing the model

In this ipynb, we load the trained models and use it to visualize how it well it would work : given a query and all the paragraphs of a wikipedia page, what would be the answer our model would provide ?

Code adapted from :
https://github.com/jonbruner/tensorflow-basics/blob/master/save-load/load.ipynb

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
# load data
#q = np.array(np.load("../data/old_data/short_fold0_600K_query_text.csv").tolist())
#p = np.array(np.load("../data/old_data/short_fold0_600K_paragraph_text.csv").tolist())
#y = np.array(np.load("../data/short_fold0_600K_labels.csv").tolist())

In [4]:
df_test = pd.read_csv('../data/short_fold0_600K_test_df.csv')

In [6]:
df_test.head()

,Unnamed: 0,pagename,section,para_id,para_text,rel_label,query,pagename_index
0,703,14th united states congress,members,62ee9c5afa296ce5602471349331fa53b6b32fc5,All representatives were elected statewide on ...,1,members of 14th united states congress,3
1,974,63rd united states congress,members,ef4dcd79d387e7be42f5f3a484f12fdc123801cf,(3-2 Republican),1,members of 63rd united states congress,9
2,1240,29th united states congress,party summary,0c7fdc96bdf4d4186d2ff0848388e58b3227672f,(1 Whig),0,party summary of 29th united states congress,5
3,1391,29th united states congress,committees,82a0a24c1c3ec62209ced189beb586ed46f6647c,(7 Democrats),0,committees of 29th united states congress,5
4,2509,63rd united states congress,committees,85c6063a8f7dc43f15edff6f1c194a15229fafa4,(6-2 Democratic),0,committees of 63rd united states congress,9


In [8]:
# embed data in order to match as it was passed to model

# exctract word2vec vectors
import spacy
nlp = spacy.load('en')

def embed(column_name):
    vecs = []
    for qu in list(df_test[column_name]):
        doc = nlp(qu) 
        mean_vec = np.zeros([len(doc), 300])
        for word in doc:   
            vec = word.vector
            # only if using tf-idf
            '''
            # fetch df score
            try:
                idf = word2tfidf[str(word)]
            except:
                #print word
                idf = 0
            '''
            # compute final vec
            mean_vec += vec # * idf
        mean_vec = mean_vec.mean(axis=0)
        vecs.append(mean_vec)
    return vecs
    
query_vecs = embed('query')
paragraph_vecs = embed('para_text')

df_test['query_CBOW'] = list(query_vecs)
df_test['paragraph_CBOW'] = list(paragraph_vecs)

In [9]:
# Load desired model (full model = graph (.meta file) + weights) 
model_path = '../runs/1491436470/checkpoints/'
sess = tf.InteractiveSession()
new_saver = tf.train.import_meta_graph(model_path+'model-38300.meta') # graph
new_saver.restore(sess, model_path+'model-38300') # weigths

In [10]:
# Visualize the graph if needed (useful to get the tensor names required for next cell)
#tf.get_default_graph().as_graph_def()

In [17]:
# Set the variables we are going to need to give to the model to interact with it 
# Make sure the names match with the node names in the saved tf graph (possible to visualize it in previous cell)
query = sess.graph.get_tensor_by_name("input_q:0")
paragraph = sess.graph.get_tensor_by_name("input_p:0")
#scores = sess.graph.get_tensor_by_name("output/scores:0")
predictions = sess.graph.get_tensor_by_name("output/predictions:0")
dropout_keep_prob = sess.graph.get_tensor_by_name("dropout_keep_prob:0")

In [18]:
# Reshape to match required shapes
p_test = np.reshape(df_test['paragraph_CBOW'][0], (1,300))
q_test = np.reshape(df_test['query_CBOW'][0], (1,300))

In [ ]:
# TO DO

# SAVE DICTIONNARY AND EMBEDDING MATRICE WHEN RUNNING MODEL?

# OR IN THE MODEL, ONCE EVERY X EPOCHS, PASS ALL DATA THAT HAS ID Y AND TRY TO RECONSTRUCT WIKIPEDIA ARTICLE

In [19]:
# Make prediction
#result = sess.run(scores, feed_dict={paragraph:p_test, query:q_test, dropout_keep_prob:1})
result = sess.run(predictions, feed_dict={paragraph:p_test, query:q_test, dropout_keep_prob:1})
#print('scores :')
#print(result)
print('model prediction :')
#print(sess.run(tf.argmax(result, 1))[0])
print(sess.run(result))
print('true prediction :')
print(np.argmax(y[0]))

ValueError: Cannot feed value of shape (1, 300) for Tensor 'input_p:0', which has shape '(?, 1000)'